In [1]:
import scvi
import os

import scanpy as sc
import scvelo as scv
#import scib
#MemoryError: Cannot allocate write+execute memory for ffi.callback().  
#You might be running on a system that prevents this. 
#For more information, see https://cffi.readthedocs.io/en/latest/using.html#callbacks

# I think it is rpy2's problem

import matplotlib.pyplot as plt
import time

import numpy as np

#sc.set_figure_params(figsize=(6,6))

Global seed set to 0
/Users/cindywei/miniconda3/envs/scvi/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/Users/cindywei/miniconda3/envs/scvi/lib/python3.9/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [5]:
np.random.seed(123)

In [3]:
from pathlib import Path, PurePath
loc_dir = Path('.')

In [6]:
# let's hack the scib
import silhouette
import nmi
import ari
import lisi

In [8]:
# remove weird grid from scvelo
plt.rcParams['axes.grid'] = False

In [9]:
print(plt.rcParams['axes.prop_cycle'])

cycler('color', ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])


data_folder = '/fast/users/twei_m/work/crc/datasets'

In [10]:
adata_concat = sc.read(Path('adata_concat_with_joanito.h5'))

In [11]:
adata = adata_concat.copy()

## load model

In [12]:
# load models
model_path_list = list(loc_dir.glob('model_20230525*icms*'))

In [13]:
model_path_list

[PosixPath('model_20230525_172203_icms_source_pt_mito')]

In [12]:
# add model latent space to adata
def add_latent_space(model_path, adata):
    model = scvi.model.SCVI.load(model_path, adata=adata, use_gpu=False)
    print(model, 'reconstruction error', model.get_reconstruction_error())
    latent = model.get_latent_representation()
    adata.obsm["X_scVI_"+ PurePath(model_path).name] = latent

In [13]:
add_latent_space(model_path_list[0], adata)

INFO     File model_20221206_153705_icms_pt_mito/model.pt already downloaded                                       


SCVI Model with the following params: 
n_hidden: 128, n_latent: 10, n_layers: 1, dropout_rate: 0.1, dispersion: gene, gene_likelihood: zinb, 
latent_distribution: normal
Training status: Trained

 reconstruction error {'reconstruction_loss': -1177.57240342986}


### Latent space UMAP

In [15]:
cat_covariate_list = ["patient"]
cont_covariate_list = ["percent_mito"]

In [16]:
# use scVI latent space for UMAP generation
def plot_latent(model_type, model_name, model_path, cat_covariate_or_batch, celltype_label_colname, 
                evaluation):

        #rep = "X_scVI_"+ PurePath(model_path).name
    if model_path:
        rep = model_type + PurePath(model_path).name
    elif model_name:
        rep = model_type + model_name
    else:
        rep = model_type
    print(rep)
    
    adata_scvi = sc.pp.neighbors(adata, use_rep = rep, copy=True)
    sc.tl.umap(adata_scvi)
    sc.tl.louvain(adata_scvi, key_added='louvain_integrated1', resolution=1)
    sc.tl.louvain(adata_scvi, key_added='louvain_integrated2', resolution=2)
    sc.tl.louvain(adata_scvi, key_added='louvain_integrated5', resolution=5)
    if model_type == 'X_scANVI':
        scv.pl.scatter(adata_scvi, basis='umap', color=['sample_origin', 'patient', 'MS_status', 
                                                    celltype_label_colname,
                                                    rep.replace('X_', 'C_'),
                                                       'numbat',
                                                       'CCISM_call',
                                                       'source'], 
               ncols=2, dpi=150, legend_loc='right margin')
    else:
        scv.pl.scatter(adata_scvi, basis='umap', color=['sample_origin', 'patient', 'MS_status', 
                                                    celltype_label_colname], 
               ncols=2, dpi=150, legend_loc='right margin')

    # integration evaluation
    if evaluation == True:
        # NMI: overlap of two clusterings e.g. cell type label vs louvain
        print('NMI: ' ,nmi.nmi(adata_scvi, celltype_label_colname, 'louvain_integrated1'),
             nmi.nmi(adata_scvi, celltype_label_colname, 'louvain_integrated2'),
             nmi.nmi(adata_scvi, celltype_label_colname, 'louvain_integrated5'))
        # ARI: overlap of two clusterings considering disagreement
        print('ARI: ' ,ari.ari(adata_scvi, celltype_label_colname, 'louvain_integrated1'),
             ari.ari(adata_scvi, celltype_label_colname, 'louvain_integrated2'),
             ari.ari(adata_scvi, celltype_label_colname, 'louvain_integrated5'))
        # Silhouette, ASW: within cluster dist vs between cluster dist (closest cluster) of a cell
        print('silhouette score: ', silhouette.silhouette(adata, celltype_label_colname, rep))
        print('silhouette batch score: ' , silhouette.silhouette_batch(adata, cat_covariate_or_batch, 
                                                                            celltype_label_colname, rep))


In [ ]:
# plot UMAP and shilhouette score
plot_latent(model_type='X_scVI_', 
                model_name=None,
                model_path=model_path_list[0], 
                cat_covariate_or_batch=cat_covariate_list[0], 
                celltype_label_colname='cell_type_epi_custom_simplified_nonan', 
                evaluation=False)

## scANVI label transfer

In [52]:
def run_scavi_and_add_latent_and_priumap(model_path, cell_type_label, save_model):
    
    model = scvi.model.SCVI.load(model_path, adata=adata, use_gpu=False)
    
    lvae = scvi.model.SCANVI.from_scvi_model(
            model,
            adata=adata,
            unlabeled_category='Unknown',
            labels_key=cell_type_label)
    lvae.train(max_epochs=20)
    
    print(lvae, 'reconstruction error', lvae.get_reconstruction_error())
    
    if save_model == True:
        lvae.save(Path(PurePath(model_path).name + '_scANVI'+ time.strftime("_20%y%m%d_%H%M%S", time.localtime())))
    
    print("C_scANVI_"+PurePath(model_path).name)
    adata.obs["C_scANVI_"+PurePath(model_path).name] = lvae.predict(adata)
    adata.obsm["X_scANVI_"+PurePath(model_path).name] = lvae.get_latent_representation(adata)
    
    scv.pl.scatter(adata, basis='umap', color=["C_scANVI_"+PurePath(model_path).name, 
                                               cell_type_label], 
               ncols=2, dpi=150, legend_loc='right margin', size = 5)

In [ ]:
for i in model_path_list:
    lvae = run_scavi_and_add_latent_and_priumap(i, 'iCMS', True)

### fix some colors

### pt + mito

In [62]:
adata.uns['icms_cluster_new_pub_up_by_cell_colors'] = ['#1f77b4', '#ff7f0e', '#279e68']

In [57]:
adata.obs['C_scANVI_model_20221206_153705_icms_pt_mito'] = adata.obs['C_scANVI_model_20221206_153705_icms_pt_mito'].cat.reorder_categories(['iCMS2', 'iCMS3', 'Normal'])

In [60]:
adata.uns['C_scANVI_model_20221206_153705_icms_pt_mito_colors'] = ['#1f77b4', '#ff7f0e', '#279e68']

In [ ]:
scv.pl.scatter(adata, basis='umap', color=['C_scANVI_model_20221206_153705_icms_pt_mito'], 
               ncols=2, dpi=150, legend_loc='right margin', size = 5)

### some evaluation of integration

In [ ]:
plot_latent(model_type='X_scANVI', 
                model_name='_model_20221206_153705_icms_pt_mito',
                model_path=None, 
                cat_covariate_or_batch=cat_covariate_list[0], 
                celltype_label_colname='iCMS', 
                evaluation=False)

### confusion matrix: label changes

### pt + mito

In [66]:
def confusion_mtx(adata, observed_col, prediction_col):
    
    df = adata.obs.groupby([observed_col,
                            prediction_col]).size().unstack(fill_value=0)
    conf_mtx=df/(df.sum(axis=1).values[:, np.newaxis])
    
    plt.figure(figsize=(8,8))
    _ = plt.pcolor(conf_mtx)
    _ = plt.xticks(np.arange(0.5, len(df.columns),1), df.columns, rotation=90)
    _ = plt.yticks(np.arange(0.5, len(df.index),1), df.index)
    plt.xlabel('Predicted')
    plt.ylabel('Observed')
    plt.title(prediction_col)

In [ ]:
confusion_mtx(adata,'iCMS', 'C_scANVI_model_20221206_153705_icms_pt_mito')

In [ ]:
adata.write(Path('CB_epi_cells_iCMS.h5'))